## Importar librerias

In [30]:
import pandas as pd
import pyodbc
import sqlite3
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
# -------------------------
# CONFIGURACIÓN DE CONEXIONES
# -------------------------

# Conexión a Azure SQL Database
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'
azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"
conn = pyodbc.connect(azure_conn_str)

In [18]:
# Ejecutar consulta
query = "SELECT * FROM [DATAEX].[IA_Client_segment_MODIFICADO]"
df = pd.read_sql_query(query, conn)

# Exportar a CSV
df.to_csv("IA_Client.csv", index=False)

# Cerrar la conexión
conn.close()

C:\Users\jaal2\AppData\Local\Temp\ipykernel_4904\3354805586.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [27]:
df = pd.read_csv("IA_Client.csv", sep=",", encoding="utf-8")
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385.0,80,True,195,NaN
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052.0,50,True,360,5.0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717.0,30,False,407,3.0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606.0,30,False,544,4.0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346.0,30,True,170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424.0,30,True,106,NaN
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514.0,50,False,257,NaN
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385.0,80,True,347,5.0
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974.0,30,False,582,1.0


In [25]:
df.dtypes

ID                            object
Genero                        object
Casado                        object
Edad                           int64
Graduado                      object
Profesion                     object
Experiencia_laboral           object
Gastoscore                    object
Family_Size                    int64
anno_nacimiento                int64
Generacion                    object
Ingresos_anuales_brutos        int64
Provincia                     object
Digital_encuesta             float64
Gasto_medio_mensual_cuota      int64
Abandono                        bool
Dias_cliente                   int64
Campanna_anno                float64
dtype: object

In [26]:
df.isnull().sum()

ID                              0
Genero                          0
Casado                        190
Edad                            0
Graduado                      103
Profesion                       0
Experiencia_laboral             0
Gastoscore                      0
Family_Size                     0
anno_nacimiento                 0
Generacion                      0
Ingresos_anuales_brutos         0
Provincia                       0
Digital_encuesta                0
Gasto_medio_mensual_cuota       0
Abandono                        0
Dias_cliente                    0
Campanna_anno                5707
dtype: int64

In [28]:
df['Campanna_anno'] = df['Campanna_anno'].fillna(0)
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385.0,80,True,195,0.0
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052.0,50,True,360,5.0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717.0,30,False,407,3.0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606.0,30,False,544,4.0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346.0,30,True,170,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424.0,30,True,106,0.0
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514.0,50,False,257,0.0
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385.0,80,True,347,5.0
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974.0,30,False,582,1.0


In [29]:
df = df.dropna(subset=['Graduado', 'Casado'])
df.isnull().sum()

ID                           0
Genero                       0
Casado                       0
Edad                         0
Graduado                     0
Profesion                    0
Experiencia_laboral          0
Gastoscore                   0
Family_Size                  0
anno_nacimiento              0
Generacion                   0
Ingresos_anuales_brutos      0
Provincia                    0
Digital_encuesta             0
Gasto_medio_mensual_cuota    0
Abandono                     0
Dias_cliente                 0
Campanna_anno                0
dtype: int64

## Standar Scaler, Label Encoder, One Hot Encoder

In [32]:
df['Provincia'].unique()

array(['Segovia', 'Alicante', 'Toledo', 'Leon', 'LLeida', 'Ciudad Real',
       'i\x81vila', 'Guipiºzcoa', 'Vizcaya', 'Orense', 'Castellon',
       'Pontevedra', 'Almeri\xada', 'Valladolid', 'Jaen', 'Lugo',
       'Santa Cruz de Tenerife', 'Albacete', 'Huelva', 'Salamanca',
       'Ceuta', 'Baleares', 'Gerona', 'Barcelona', 'Sevilla',
       'Guadalajara', 'Soria', 'Caceres', 'Madrid', 'Cordoba', 'Burgos',
       'Navarra', 'Melilla', 'Granada', 'Badajoz', 'Valencia',
       'Tarragona', 'La Corui±a', 'Las Palmas', 'Murcia', 'Palencia',
       'La Rioja', 'Teruel', 'Zamora', 'Zaragoza', 'i\x81lava', 'Cuenca',
       'Huesca', 'Cadiz', 'Malaga'], dtype=object)

In [37]:
# Diccionario que mapea cada Comunidad Autónoma con las provincias (exactamente como aparecen en la lista)
comunidades = {
    'Andalucía': ['Almeri\xada', 'Jaen', 'Huelva', 'Sevilla', 'Cordoba', 'Granada', 'Cadiz', 'Malaga'],
    'Aragón': ['Huesca', 'Teruel', 'Zaragoza'],
    'Castilla-La Mancha': ['Toledo', 'Ciudad Real', 'Albacete', 'Guadalajara', 'Cuenca'],
    'Castilla y León': ['Segovia', 'Leon', 'i\x81vila', 'Valladolid', 'Salamanca', 'Soria', 'Palencia', 'Burgos', 'Zamora'],
    'Cataluña': ['LLeida', 'Gerona', 'Barcelona', 'Tarragona'],
    'Extremadura': ['Caceres', 'Badajoz'],
    'Galicia': ['Orense', 'Pontevedra', 'Lugo', 'La Corui±a'],
    'Comunidad Valenciana': ['Alicante', 'Castellon', 'Valencia'],
    'Comunidad de Madrid': ['Madrid'],
    'Navarra': ['Navarra'],
    'País Vasco': ['Guipiºzcoa', 'Vizcaya', 'i\x81lava'],
    'Baleares': ['Baleares'],
    'Canarias': ['Santa Cruz de Tenerife', 'Las Palmas'],
    'Murcia': ['Murcia'],
    'La Rioja': ['La Rioja'],
    'Ceuta': ['Ceuta'],
    'Melilla': ['Melilla']
}

In [38]:
def assign_communidad(region):
    # Se asume que en 'region' las provincias están separadas por ';'
    provincias = [provincia.strip() for provincia in region.split(';')]
    communidades = set()
    
    for province in provincias:
        for community, provinces_list in comunidades.items():
            if province in provinces_list:
                communidades.add(community)
    
    # Devolver la lista de comunidades como cadena, separadas por coma
    return ', '.join(communidades) if communidades else None

# Aplicamos la función a la columna que contenga las provincias y creamos una nueva columna
df['comunidad'] = df['Provincia'].apply(assign_communidad)

C:\Users\jaal2\AppData\Local\Temp\ipykernel_4904\4163525337.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comunidad'] = df['Provincia'].apply(assign_communidad)


In [40]:
provincia_sin_comunidad = df[df['comunidad'].isnull()]['Provincia'].unique()

# Imprimimos los países que no tienen continente asignado
print("Provincias sin comunidad autónoma asignada:")
for country in provincia_sin_comunidad:
    print(country)

Provincias sin comunidad autónoma asignada:


In [41]:
df_null_continent = df[df['comunidad'].isnull()]

In [42]:
def assign_communidad(region):
    # Se asume que en 'region' las provincias están separadas por ';'
    provincias = [provincia.strip() for provincia in region.split(';')]
    communidades = set()
    
    for province in provincias:
        for community, provinces_list in comunidades.items():
            if province in provinces_list:
                communidades.add(community)
    
    # Devolver la lista de comunidades como cadena, separadas por coma
    return ', '.join(communidades) if communidades else None

In [43]:
df['comunidad'].str.split(',').explode().str.strip().unique()

array(['Castilla y León', 'Comunidad Valenciana', 'Castilla-La Mancha',
       'Cataluña', 'País Vasco', 'Galicia', 'Andalucía', 'Canarias',
       'Ceuta', 'Baleares', 'Extremadura', 'Comunidad de Madrid',
       'Navarra', 'Melilla', 'Murcia', 'La Rioja', 'Aragón'], dtype=object)

In [44]:
df_exploded = df['comunidad'].str.split(',').explode().str.strip()
comunidad_dummies = pd.get_dummies(df_exploded)
df = pd.concat([df, comunidad_dummies], axis=1)

In [45]:
df.drop('Provincia', axis=1, inplace=True)

In [49]:
# Lista de columnas que corresponden a las comunidades
community_columns = [
    'Andalucía', 'Aragón', 'Castilla-La Mancha', 'Castilla y León', 
    'Cataluña', 'Extremadura', 'Galicia', 'Comunidad Valenciana', 
    'Comunidad de Madrid', 'Navarra', 'País Vasco', 'Baleares', 
    'Canarias', 'Murcia', 'La Rioja', 'Ceuta', 'Melilla'
]

# Convertir las columnas booleanas a enteros (True->1, False->0)
df[community_columns] = df[community_columns].astype(int)

# Mostrar las primeras filas para verificar la conversión

df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,...,Ceuta,Comunidad Valenciana,Comunidad de Madrid,Extremadura,Galicia,La Rioja,Melilla,Murcia,Navarra,País Vasco
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,...,0,0,0,0,0,0,0,0,0,0
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,...,0,1,0,0,0,0,0,0,0,0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,...,0,0,0,0,0,0,0,0,0,0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,...,0,0,0,0,0,0,0,0,0,0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,...,0,0,0,0,0,0,0,0,0,0
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,...,0,0,0,0,0,0,0,0,0,0
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,...,0,0,0,0,1,0,0,0,0,0
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df.head(2)

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,...,Ceuta,Comunidad Valenciana,Comunidad de Madrid,Extremadura,Galicia,La Rioja,Melilla,Murcia,Navarra,País Vasco
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,...,0,0,0,0,0,0,0,0,0,0
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
# Definir las columnas para One Hot Encoding
onehot_cols = ['Profesion']

#for col in onehot_cols:
#    ohe = OneHotEncoder(sparse=False, drop='first')
#    encoded = ohe.fit_transform(df[[col]])
    # Crear nombres de las nuevas columnas (se omite la primera categoría por drop='first')
#    new_cols = [f"{col}_{cat}" for cat in ohe.categories_[0][1:]]
#    df_ohe = pd.DataFrame(encoded, columns=new_cols, index=df.index)
#    # Concatenar las columnas codificadas y eliminar la original
#    df = pd.concat([df.drop(col, axis=1), df_ohe], axis=1)

#print("One Hot Encoding completado. Nuevas columnas:")
#print(df.columns)


In [52]:
# Lista de columnas para Label Encoding
label_cols = ['Genero', 'Casado', 'Graduado', 'Digital_encuesta', 'Abandono', 'Generacion', 'Campanna_anno', 'Profesion']

for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,...,Ceuta,Comunidad Valenciana,Comunidad de Madrid,Extremadura,Galicia,La Rioja,Melilla,Murcia,Navarra,País Vasco
0,CLI10,0,1,67,1,4,+20annos,Bajo,3,1957,...,0,0,0,0,0,0,0,0,0,0
1,CLI100,1,1,64,1,5,+20annos,Medio,2,1960,...,0,1,0,0,0,0,0,0,0,0
2,CLI1000,1,1,65,1,0,+20annos,Medio,3,1959,...,0,0,0,0,0,0,0,0,0,0
3,CLI10000,1,0,33,0,3,10-20annos,Bajo,4,1991,...,0,0,0,0,0,0,0,0,0,0
4,CLI10001,1,1,52,0,7,+20annos,Medio,2,1972,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,0,1,76,0,8,+20annos,Alto,2,1948,...,0,0,0,0,0,0,0,0,0,0
10802,CLI9996,0,0,44,1,3,+20annos,Bajo,1,1980,...,0,0,0,0,0,0,0,0,0,0
10803,CLI9997,1,1,47,1,2,+20annos,Alto,3,1977,...,0,0,0,0,1,0,0,0,0,0
10804,CLI9998,1,1,40,1,0,10-20annos,Alto,3,1984,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# Lista de columnas numéricas a escalar
scale_cols = ['Edad', 'Experiencia_laboral', 'Family_Size', 'anno_nacimiento', 
              'Ingresos_anuales_brutos', 'Gasto_medio_mensual_cuota', 'Dias_cliente']

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

df

ValueError: could not convert string to float: '+20annos'

In [ ]:
# Definir columnas ordinales y el orden de categorías para cada una
ordinal_cols = ['Gastoscore']
categories = [
    ['Bajo', 'Medio', 'Alto']
]

ord_encoder = OrdinalEncoder(categories=categories)
df[ordinal_cols] = ord_encoder.fit_transform(df[ordinal_cols])

print("Ordinal Encoding completado. Ejemplo de datos:")
print(df[ordinal_cols].head())